In [1]:
from IPython.display import display
import ipywidgets as widgets

from src.hybrid_search import hybrid_search, rrf_fusion
from src.reranker import rerank_results

# UI elements
query_box = widgets.Text(
    value='',
    placeholder='Type your search query here',
    description='Query:',
    layout=widgets.Layout(width='100%')
)

user_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='User ID:'
)

output = widgets.Output()

In [2]:
def run_search(query, user_id):
    dense_results, sparse_results = hybrid_search(query, user_id)
    fused = rrf_fusion([dense_results, sparse_results])[:10]
    
    docs = [
        hit.payload['title'] + ' ' + hit.payload.get('overview', '')
        for hit in fused
    ]
    scores = rerank_results(query, docs)

    with output:
        output.clear_output()
        for i, (hit, score) in enumerate(zip(fused, scores), 1):
            print(f"{i}. {hit.payload['title']} (Score: {score:.4f})")


In [ ]:
search_button = widgets.Button(description="🔍 Search")

def on_button_clicked(_):
    run_search(query_box.value, user_slider.value)

search_button.on_click(on_button_clicked)

display(widgets.VBox([
    query_box,
    user_slider,
    search_button,
    output
]))